# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 0.9769614390770253),
 (1, 0.9769614390770253),
 (2, 0.9769614390770253),
 (3, 0.9769614390770253),
 (4, 0.9769614390770253)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('is', 18), ('it', 18)]),
 (1, [('what', 10)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.015986376939784774),
   (None, 0.05486345639886647),
   (None, 0.06482719250038715),
   (None, 0.09615622339126895),
   (None, 0.10073695540273941),
   (None, 0.114977758408776),
   (None, 0.1713227259284985),
   (None, 0.18848976964912512),
   (None, 0.2702472503905019),
   (None, 0.27163862200310807),
   (None, 0.35715021262817936),
   (None, 0.4106340739687878),
   (None, 0.4255751963739316),
   (None, 0.46163373194468216),
   (None, 0.4928270684919176),
   (None, 0.49303993781020927)]),
 (1,
  [(None, 0.5416508451124052),
   (None, 0.6032000500678181),
   (None, 0.6285894338194947),
   (None, 0.6564315138849527),
   (None, 0.823570750228809),
   (None, 0.8522754097109102),
   (None, 0.8886692037059769),
   (None, 0.8971347038131579),
   (None, 0.8984985552536929),
   (None, 0.9361725443827631),
   (None, 0.944881849114131),
   (None, 0.9512141052186531),
   (None, 0.9516889159131262),
   (None, 0.996634671980391)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

input_values = np.random.rand(30)

def MAP(_, row:int):
  yield (None, row)

def REDUCE(_, rows:list):
  yield (_, max(rows))

def RECORDREADER():
  return [(None, u) for u in input_values]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(None, 0.9925199502283611)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

input_values = np.random.rand(30)

def MAP(_, row:int):
  yield (None, row)

def REDUCE(_, rows:list):
  yield (_, sum(rows)/len(rows))

def RECORDREADER():
  return [(None, u) for u in input_values]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(None, 0.4940336516302871)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
data_list = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

#{key: [...values]}

def RECORDREADER(key:str):
  for user in data_list:
    yield (getattr(user, key), user)

def MAP(idx:int, value:int):
   yield (idx, value)

def groupByKey(iterable):
  t = {}
  #print(list(iterable))
  for k2, v2 in sorted(iterable, key=lambda x:x[0]):
   # print(key, value)
    if k2 not in t.keys():
      t[k2] = []
    t[k2] = t.get(k2, []) + [v2]
  return t.items()


print(list(groupByKey(flatten(map(lambda x: MAP(*x), RECORDREADER("id"))))), "\n")

print(list(groupByKey(flatten(map(lambda x: MAP(*x), RECORDREADER("gender"))))), "\n")

print(list(groupByKey(flatten(map(lambda x: MAP(*x), RECORDREADER("age"))))))

[(0, [User(id=0, age=55, social_contacts=20, gender='male')]), (1, [User(id=1, age=25, social_contacts=240, gender='female')]), (2, [User(id=2, age=25, social_contacts=500, gender='female')]), (3, [User(id=3, age=33, social_contacts=800, gender='female')])] 

[('female', [User(id=1, age=25, social_contacts=240, gender='female'), User(id=2, age=25, social_contacts=500, gender='female'), User(id=3, age=33, social_contacts=800, gender='female')]), ('male', [User(id=0, age=55, social_contacts=20, gender='male')])] 

[(25, [User(id=1, age=25, social_contacts=240, gender='female'), User(id=2, age=25, social_contacts=500, gender='female')]), (33, [User(id=3, age=33, social_contacts=800, gender='female')]), (55, [User(id=0, age=55, social_contacts=20, gender='male')])]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
import numpy as np

data_list = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=3, age=33, gender='female', social_contacts=800)
]


maps = 3
reducers = 1

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for u in split:
      yield ((u.id), u)

  split_size =  int(np.ceil(len(data_list)/maps))
  #print(split_size)
  for i in range(0, len(data_list), split_size):
    yield RECORDREADER(data_list[i:i+split_size])

def MAP(userId:str, user:User):
  yield (userId, user)

def REDUCE(_, users:list):
  yield list(set(users))

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

9 key-value pairs were sent over a network.


[(0,
  [[User(id=0, age=55, social_contacts=20, gender='male')],
   [User(id=1, age=25, social_contacts=240, gender='female')],
   [User(id=3, age=33, social_contacts=800, gender='female')]])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=3, age=33, gender='female', social_contacts=800)
]


def MAP(_, row:NamedTuple):
  if (row.social_contacts > 100):
    yield (row, row)

def REDUCE(_, rows:list):
  yield rows

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[[User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')],
 [User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')]]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [31]:
class Projection(NamedTuple):
  id:int
  age:int
  gender:str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=3, age=33, gender='female', social_contacts=800)
]


def MAP(_, row:NamedTuple):
  pr_user = Projection(id=row.id,age=row.age,gender=row.gender)
  yield (pr_user, pr_user)

def REDUCE(row, rows:list):
  yield (row,row)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(Projection(id=0, age=55, gender='male'),
  Projection(id=0, age=55, gender='male')),
 (Projection(id=1, age=25, gender='female'),
  Projection(id=1, age=25, gender='female')),
 (Projection(id=3, age=33, gender='female'),
  Projection(id=3, age=33, gender='female'))]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [46]:
input_collection_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
  User(id=2, age=21, gender='female', social_contacts=270),
  User(id=4, age=33, gender='female', social_contacts=40),
  User(id=8, age=44, gender='male', social_contacts=400),
  User(id=16, age=37, gender='male', social_contacts=660),
  User(id=32, age=11, gender='female', social_contacts=540)
]

input_collection = [input_collection_1,input_collection_2]

def MAP(_, row:NamedTuple):
  yield (row, row)

def REDUCE(row, rows:list):
  yield (row,row)

def RECORDREADER():
  return [(u.id, u) for input_col in input_collection for u in input_col]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')),
 (User(id=2, age=21, social_contacts=270, gender='female'),
  User(id=2, age=21, social_contacts=270, gender='female')),
 (User(id=4, age=33, social_contacts=40, gender='female'),
  User(id=4, age=33, social_contacts=40, gender='female')),
 (User(id=8, age=44, social_contacts=400, gender='male'),
  User(id=8, age=44, social_contacts=400, gender='male')),
 (User(id=16, age=37, social_contacts=660, gender='male'),
  User(id=16, age=37, social_contacts=660, gender='male')),
 (User(id=32, age=11, social_contacts=540, gender='female'),
  User(id=32, age=11, social_contacts=540, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [48]:
input_collection_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=21, gender='female', social_contacts=270),
    User(id=4, age=33, gender='female', social_contacts=40),
    User(id=8, age=44, gender='male', social_contacts=400),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
  User(id=2, age=21, gender='female', social_contacts=270),
  User(id=4, age=33, gender='female', social_contacts=40),
  User(id=8, age=44, gender='male', social_contacts=400),
  User(id=16, age=37, gender='male', social_contacts=660),
  User(id=32, age=11, gender='female', social_contacts=540)
]

input_collection = [input_collection_1,input_collection_2]

def MAP(_, row:NamedTuple):
  yield (row, row)

def REDUCE(row, rows:list):
  if len(rows) == 2:
    yield (row,row)

def RECORDREADER():
  return [(u.id, u) for input_col in input_collection for u in input_col]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=2, age=21, social_contacts=270, gender='female'),
  User(id=2, age=21, social_contacts=270, gender='female')),
 (User(id=4, age=33, social_contacts=40, gender='female'),
  User(id=4, age=33, social_contacts=40, gender='female')),
 (User(id=8, age=44, social_contacts=400, gender='male'),
  User(id=8, age=44, social_contacts=400, gender='male'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [66]:
input_collection_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=21, gender='female', social_contacts=270),
    User(id=4, age=33, gender='female', social_contacts=40),
    User(id=8, age=44, gender='male', social_contacts=400),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
  User(id=2, age=21, gender='female', social_contacts=270),
  User(id=4, age=33, gender='female', social_contacts=40),
  User(id=8, age=44, gender='male', social_contacts=400),
  User(id=16, age=37, gender='male', social_contacts=660),
  User(id=32, age=11, gender='female', social_contacts=540)
]

input_collection = [input_collection_1,input_collection_2]

def MAP(R, row:NamedTuple):
  yield (row, R)

def REDUCE(row, rows:list):
  if len(rows) == 1 and rows[0]==0:
    yield (row,row)

def RECORDREADER():
  return [(id, u) for id, input_col in enumerate(input_collection) for u in input_col]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [86]:
class IdGender(NamedTuple):
  id:int
  gender: str

class IdSocialContact(NamedTuple):
  id:int
  social_contacts: int

input_collection_1 = [
    IdGender(id=0, gender='male'),
    IdGender(id=0, gender='female'),
    IdGender(id=1, gender='female'),
    IdGender(id=2, gender='female')
]

input_collection_2 = [
  IdSocialContact(id=0, social_contacts=270),
  IdSocialContact(id=0, social_contacts=112),
  IdSocialContact(id=1, social_contacts=40),
  IdSocialContact(id=2, social_contacts=400)
]

input_collection = [input_collection_1,input_collection_2]

def MAP(R, row:NamedTuple):
  yield (row.id, (row, R))

def REDUCE(rowId, rows:list):
  a_data = [row[0].gender for row in rows if row[1] ==0]
  c_data = [row[0].social_contacts for row in rows if row[1] ==1]
  for a in a_data:
    for c in c_data:
      yield (rowId, a, c)

def RECORDREADER():
  return [(id, u) for id, input_col in enumerate(input_collection) for u in input_col]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 'male', 270),
 (0, 'male', 112),
 (0, 'female', 270),
 (0, 'female', 112),
 (1, 'female', 40),
 (2, 'female', 400)]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [98]:
class UserSalary(NamedTuple):
  id:int
  salary:int
  gender:str


input_collection_1 = [
  UserSalary(id=0, gender='male', salary=210),
  UserSalary(id=0, gender='female', salary=55),
  UserSalary(id=1, gender='female', salary=370),
  UserSalary(id=1, gender='female', salary=670)
]

input_collection_2 = [
  UserSalary(id=0, gender='male', salary=270),
  UserSalary(id=0, gender='female', salary=112),
  UserSalary(id=1, gender='female', salary=40),
  UserSalary(id=1, gender='female', salary=400)
]


input_collection = [input_collection_1,input_collection_2]

def RECORDREADER():
  return [(u.id, u) for input_col in input_collection for u in input_col]

def MAP(userID:int, user:UserSalary):
  yield (userID, user.salary)

def REDUCE(userId:int, data:list):
  yield (userId, max(data))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 270), (1, 670)]

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [122]:
import numpy as np

mat = np.ones((4, 4))
vec = np.array([2, 4, 1, 1])

maps = 3
reducers = 1

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for i in range(split.shape[0]):
      for j in range(split.shape[1]):
        yield (i, (split[i, j], vec[j],split.shape[1]))

  split_size = int(np.ceil(len(mat) / maps))

  for i in range(0, len(mat), split_size):
    yield RECORDREADER(mat[i: i + split_size])


def MAP(coordinates, values):
  i = coordinates
  mat_val, vec_val, split_size = values
  yield (split_size, mat_val * vec_val)


def REDUCE(sp, products:Iterator[NamedTuple]):
  split_size = sp
  yield (sum(products)/split_size, sum(products)/split_size)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

16 key-value pairs were sent over a network.


[(0, [(8.0, 8.0)])]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [27]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [128]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for row in range(small_mat.shape[0]):
    yield ((row, k), w * small_mat[row][j])
  # solution code that yield(k2,v2) pairs

def REDUCE(key, values):
  (i, k) = key
  yield(key,sum(values))
  # solution code that yield(k3,v3) pairs

Проверьте своё решение

In [129]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [130]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [136]:
def RECORDREADER():
  for k in range(big_mat.shape[1]):
    for i in range(small_mat.shape[0]):
      for j in range(small_mat.shape[1]):
        yield ((i, k), (0, small_mat[i, j]))

  for i in range(small_mat.shape[0]):
    for j in range(big_mat.shape[0]):
      for k in range(big_mat.shape[1]):
        yield ((i, k), (1, big_mat[j, k]))


def MAP(k1, v1):
  yield (k1, v1)


def REDUCE(k2, v2):
  yield (k2, np.dot([val for id, val in v2 if id == 0], [val for id, val in v2 if id == 1]))


output = MapReduce(RECORDREADER, MAP, REDUCE)

np.allclose(reference_solution, asmatrix(output)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [167]:
def INPUTFORMAT():
  global maps
  def RECORDREADER_SMALL(split, idx):
    for k in range(big_mat.shape[1]):
      for i in range(split.shape[0]):
        for j in range(split.shape[1]):
          yield ((i + idx, k), (0, split[i, j]))


  split_size = int(np.ceil( small_mat.shape[0] / maps))
  for i in range(0, small_mat.shape[0], split_size):
    yield RECORDREADER_SMALL(small_mat[i : i + split_size], i)


  def RECORDREADER_BIG(split):
    for i in range(small_mat.shape[0]):
      for j in range(split.shape[0]):
        for k in range(split.shape[1]):
          yield ((i, k), (1, split[j, k]))


  split_size = int(np.ceil(big_mat.shape[0] / maps))
  for i in range(0, big_mat.shape[0], split_size):
    yield RECORDREADER_BIG(big_mat[i : i + split_size])

def distributed(result):
  rs = [value for _,values in result for value in values]
  return asmatrix((k, v) for k, v in set(rs))

def MAP(k1, v1):
  yield (k1, v1)


def REDUCE(k2, v2):
  yield (k2, np.dot([val for id, val in v2 if id == 0], [val for id, val in v2 if id == 1]))


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]


np.allclose(reference_solution, distributed(partitioned_output)) # should return true

480 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [171]:
maps = 2
reducers = 1


def INPUTFORMAT():
  global maps

  def RECORDREADER_SMALL(split, idx):
    for k in range(big_mat.shape[0]):
      for i in range(split.shape[0]):
        for j in range(split.shape[1]):
          yield ((i + idx, k), (0, split[i, j]))

  def RECORDREADER_BIG(split, idx):
    for i in range(small_mat.shape[0]):
      for j in range(split.shape[0]):
        for k in range(split.shape[1]):
          yield ((i, k + idx), (1, split[j, k]))

  small_readers = {
      0: RECORDREADER_SMALL,
      1: RECORDREADER_SMALL
  }


  big_readers = {
       0: RECORDREADER_BIG,
       1: RECORDREADER_BIG
  }

  split_size_rows = int(np.ceil(small_mat.shape[0] / maps))
  split_size_cols = int(np.ceil(small_mat.shape[1] / maps))

  counter = -1
  for i in range(0, small_mat.shape[0], split_size_rows):
    for j in range(0, small_mat.shape[1], split_size_cols):
      counter += 1
      yield small_readers[counter % len(small_readers)](small_mat[i : i + split_size_rows, j : j + split_size_cols], i)


  split_size_rows = int(np.ceil(big_mat.shape[0] / maps))
  split_size_cols = int(np.ceil(big_mat.shape[1] / maps))

  counter = -1
  for i in range(0, big_mat.shape[0], split_size_rows):
    for j in range(0, big_mat.shape[1], split_size_cols):
      counter += 1
      yield big_readers[counter % len(big_readers)](big_mat[i : i + split_size_rows, j : j + split_size_cols], j)


def MAP(k1, v1):
  yield (k1, v1)


def REDUCE(k2, v2):
  yield (k2, np.dot([val for id, val in v2 if id == 0], [val for id, val in v2 if id == 1]))


output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

def distributed(result):
  rs = [value for _,values in result for value in values]
  return asmatrix((k, v) for k, v in set(rs))
  return asmatrix((k, v) for k, v in count.items())

np.allclose(reference_solution, distributed(output)) # should return true

258 key-value pairs were sent over a network.


True

Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Нет, только в том случае когда будут совпадать порядки